In [45]:
import pandas as pd
import unicodedata
import emoji
from confusable_homoglyphs import confusables
from bs4 import BeautifulSoup


In [46]:
df = pd.read_csv('dataset/youtube_comments_judol.csv')
df.head()
print(df.columns)

Index(['video_id', 'channel_name', 'komentar'], dtype='object')


In [47]:
def normalisasi_unicode(teks):
    if not isinstance(teks, str):
        teks = str(teks)

    # normalisasi unicode standar
    teks = unicodedata.normalize("NFKC", teks)

    hasil = []
    for ch in teks:
        if ord(ch) < 128:
            hasil.append(ch)
            continue
        try:
            info = confusables.is_confusable(
                ch,
                greedy=False,
                preferred_aliases=['latin']
            )
        except Exception:
            info = False

        if info:
            pengganti = None
            for entry in info:
                for h in entry.get('homoglyphs', []):
                    cand = h.get('c')
                    if (
                        isinstance(cand, str)
                        and len(cand) == 1
                        and ord(cand) < 128
                    ):
                        pengganti = cand
                        break
                if pengganti:
                    break

            hasil.append(pengganti if pengganti else ch)
        else:
            hasil.append(ch)

    return ''.join(hasil)


def preprocess_komentar(teks):
    if pd.isna(teks):
        return ""
    teks = str(teks).lower()
    teks = emoji.demojize(teks, language='id')
    teks = normalisasi_unicode(teks)

    return teks


In [48]:
df['komentar_stage1'] = df['komentar'].apply(preprocess_komentar)



In [49]:
def hapus_combining_marks(teks):
    if not isinstance(teks, str):
        teks = str(teks)

    hasil = []
    for ch in teks:
        if unicodedata.category(ch).startswith('M'):
            continue
        hasil.append(ch)

    return ''.join(hasil)


In [50]:
def hapus_tanda_baca(teks):
    if not isinstance(teks, str):
        teks = str(teks)

    tanda_baca = [
        '.', ',', '!', '?', ':', ';',
        '(', ')', '"', "'", '[', ']', '{', '}', '-','@'
    ]

    for t in tanda_baca:
        teks = teks.replace(t, ' ')

    teks = ' '.join(teks.split())
    return teks


In [51]:

def hapus_html_parsing(teks):
    """
    Menghapus seluruh tag HTML dan hanya mengambil teks.
    Aman untuk HTML berantakan.
    """
    soup = BeautifulSoup(str(teks), "html.parser")
    return soup.get_text(separator=" ", strip=True)


In [52]:
def preprocess_komentar_bersih(teks):
    teks = preprocess_komentar(teks)
    teks = hapus_combining_marks(teks)
    teks = hapus_tanda_baca(teks)
    teks = hapus_html_parsing(teks)

    return teks


In [53]:
df['komentar_clean'] = df['komentar'].apply(preprocess_komentar_bersih)

# df[['komentar', 'komentar_clean']].head()
df[['komentar', 'komentar_clean']].sample(10)


,komentar,komentar_clean
1998,Cobain mukbang daging sidat bg pasti mantep bnget,cobain mukbang daging sidat bg pasti mantep bnget
1185,sebelum ribuan,sebelum ribuan
140,Ngilerr bgttt,ngilerr bgttt
1576,-Rp2. 611.000,rp2 611 000
2573,❤❤❤,hati_merah hati_merah hati_merah
543,Manjejeh salero mancaliak Uda ko makan sup j...,manjejeh salero mancaliak uda ko makan sup jan...
2379,-garuda hoki..member bru psti jp, garuda hoki member bru psti jp
2297,makin parah sih warga didesa gua pada ketagiha...,makin parah sih warga didesa gua pada ketagiha...
1492,Kaget ada SS di Aussie 😂,kaget ada ss di aussie wajah_gembira_berurai_a...
1175,Gerce,gerce


In [57]:
df[['komentar', 'komentar_clean']].sample(10)

# df.to_csv('dataset_preprocessed.csv', index=False)

,komentar,komentar_clean
1998,Cobain mukbang daging sidat bg pasti mantep bnget,cobain mukbang daging sidat bg pasti mantep bnget
978,"asik ada ka grace,udah lucu cntik baik lagieh,...",asik ada ka grace udah lucu cntik baik lagieh ...
31,Cobain maidahhh,cobain maidahhh
972,Kalian berdua udah cocok sih,kalian berdua udah cocok sih
1657,Totalnya sebas dari harganya -2. 611.000,totalnya sebas dari harganya 2 611 000
13,anak cikarang hadir😂,anak cikarang hadir wajah_gembira_berurai_air_...
931,"<a href=""https://www.youtube.com/watch?v=WQb70...",7 23 kukira cermin itu wajah_gembira_berurai_a...
1503,"<a href=""https://www.youtube.com/watch?v=pBcSM...",24 10 mau
1544,Aku ikut sedih kak 😢😅,aku ikut sedih kak wajah_menangis wajah_tersen...
1834,abseeen,abseeen


In [60]:

# df[df["komentar"].astype(str).str.contains("<")][["komentar"]].head(20)
df[df["komentar_clean"].astype(str).str.contains("<")][["komentar_clean"]].head(20)


,komentar_clean


In [62]:
test_df = df[['komentar_clean']].dropna()
test_df.to_csv('dataset/dataset_youtube_clean.csv', index=False)